<a href="https://colab.research.google.com/github/margretran/k/blob/main/Teymisverkefni2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Teymisverkefni 2

Verkefnið er að endurbæta fallið Simplex(c,A,b) sem notar Simplex aðferðina sem skilar fjölda Simplex ítrana sem þarf við að leysa verkefni ásamt gildi á markfalli, ákvörðunabreytum og slakabreytum.

Fallið þarf að gera leyst verkefni með ótakmarkað b, þ.e.a.s. getur leyst Ax $\le$ eða $\ge$ b verkefni. Fallið notar reglu Blands við val á breytum sem koma í grunninn og fara úr grunni.

Við eigum að prófa kóðann á eftirfarandi verkefni:

c = np.array([10,-57,-9,-24])

A = np.array([[0.5,-5.5,-2.5,9],[0.5,-1.5,-0.5,1],[1,0,0,0]])

b = np.array([0,0,1])

###Skref 1
Til að byrja með vorum við með Simplex fall sem réði ekki við dæmið sem notast er við í kóðanum hér fyrir neðan. Þegar hann var keyrður þá fór hann bara í hringi með lausnina og náði aldrei einni lausn. Þetta er vegna þess að þegar verið var að velja leaving basic variable þá var jafntefli á milli, þ.e.a.s. það eru fleiri en ein breyta sem nær 0 þegar að entering basic variable er sett inn. Það getur skipt miklu máli hvaða breyta er valin til þess að fara út og það val getur breytt því hvort reikniritið fari í endalausa hringi

Þannig vissum við hvernig hægt væri að laga kóðann, en það var að nota reglu Blands til að velja leaving basic variable.

Regla Blands segir að af þeim breytum sem geta komið inn í grunninn (þær með neikvæða stuðla í markfallinu), þá eigi að velja þá breytu sem er með minnsta indexinn.

Eins með leaving variable. Það eigi að velja breytu, út frá öllum þeim sem nær 0 fyrst þegar entering breyta er sett inn, þannig að það sé sú breyta sem hefur lægsta indexinn [0].

Þegar búið var að breyta þessu í fyrri kóðanum þá náði kóðinn loks að komast að einni niðurstöðu í 7 ítrunum.

###Skref 2

Næst ætluðum við að ná að bæta kóðann þannig að hægt væri að leysa verkefnið Ax $\ge$ b en til þess ætluðum við að nota svokallað two-phase method sem talað er um í bókinni. Hér fyrir neðan útskýrum við skrefin í þeirri aðferð en það er það sem við útfærðum í kóðanum fyrir neðan.

Við bætum við tveimur föllum í viðbót við það sem við vorum búin að gera, add_artificial_variables() og two_phase_simplex()

**Step-1:	Phase-1**

a. Fyrst búum við til nýtt markfall með því að setja núll á allar upprunalegu breyturnar og -1 á allar artificial breyturnar

b. Svo notum við simplex aðferðina til að eyða artificial breytunum úr grunninum

c. Lausnin í enda fasa 1 er basic feasible solution fyrir alvöru verkefnið sem er í fasa 2.

**Step-2:	Phase-2**

a. Þegar við erum komin í fasa 2 þá notum við upprunalega markfallið og stuðlar við artificial breyturnar er 0 þannig að við getum tekið þær í burtu frá útreikingunum.

b. Þá er simplex reikniritið aftur notað til að finna bestu lausn.


Þetta var haft í huga þegar kóðinn var gerður hér fyrir neðan. Þessi kóði virkar fyrir dæmi sem eru Ax $\ge$ b en dæmið sem er notað í kóðanum er

$MinZ	=			x_1	+		x_2$

m.t.t.

$2	x_1	+		x_2	≥	4$

$x_1	+	7	x_2	≥	7$

og  $x_1,x_2≥0$

Þetta dæmi var notað því svarið við því var vitað og við fengum rétt út úr því.

Það er hægt að nota þennan kóða fyrir dæmi sem eru bæði Ax $\ge$ b og Ax $\le$ b. Það þarf bara að skrá það niður hvort eigi við með því að gera 0 fyrir Ax $\ge$ b og 1 fyrir Ax $\le$ b.



In [20]:
import numpy as np


def pivot(Table, row, col):
  Table[row,:] = Table[row,:]/Table[row,col]
  for k in [irow for irow in range(Table.shape[0]) if irow != row]:
    Table[k,:] = Table[k,:] - Table[row,:] * Table[k,col]
  return Table

def simplex(c, A, b):
    m, n = A.shape
    Table = np.zeros((m+1, n+m+1))
    Table[0,:] = np.concatenate((-c, np.zeros(m+1)))
    Table[1:,:] = np.hstack((A, np.eye(m), b.reshape((m,1))))
    basis = np.array([i for i in range(n, n+m)]) # initial basis

    num_iterations = 0  # Count the number of iterations

    while np.any(Table[0, :-1] < 0):
        # Bland's Rule for entering variable
        col_candidates = np.where(Table[0, :-1] < 0)[0]
        if col_candidates.size == 0:  # If no candidates, problem is optimal as is
            break
        col = col_candidates[0]  # Choose the smallest index for entering variable

        # Check for unboundedness
        if all(Table[1:, col] <= 0):
            raise Exception("Problem is unbounded.")

        # Bland's Rule for leaving variable
        ratios = np.array([Table[i, -1] / Table[i, col] if Table[i, col] > 0 else np.inf for i in range(1, m + 1)])
        min_ratio = np.min(ratios)

        if min_ratio == np.inf:
            raise Exception("Problem is unbounded or infeasible under Bland's Rule.")
        row_candidates = np.where(ratios == min_ratio)[0]
        basis_candidates = basis[row_candidates]
        row = row_candidates[np.argmin(basis_candidates)] + 1  # Correcting index for tableau

        Table = pivot(Table, row, col)
        basis[row - 1] = col  # Update basis
        num_iterations += 1

    x = np.zeros(n+m)
    x[basis] = Table[1:,-1]

    return num_iterations, x, Table


def add_artificial_variables(A, b):
    m, n = A.shape
    # Add slack variables to convert "<=" constraints to "="
    A_with_slack = np.hstack((A, np.eye(m)))
    # Add artificial variables
    A_with_artificial = np.hstack((A_with_slack, np.eye(m)))
    artificial_indices = list(range(n + m, n + 2 * m))
    return A_with_artificial, b, artificial_indices

def two_phase_simplex(original_c, A, b, status):
    if status == 0 :

      # Phase 1: Add artificial variables to find a feasible basis
      A_extended, b, artificial_indices = add_artificial_variables(A, b)
      c_phase1 = np.zeros(A_extended.shape[1])
      for index in artificial_indices:
          c_phase1[index] = -1  # Minimize the sum of artificial variables

      # Solve Phase 1
      num_iterations, _, Table_phase1 = simplex(c_phase1, A_extended, b)

      # Check if solution is feasible
      if any(Table_phase1[0, artificial_indices] < -1e-5):
          return "Problem is infeasible.", None

      # Phase 2: Use the original objective with the basis found in Phase 1
      # Remove artificial variable columns from Table_phase1
      n_original = len(original_c)
      A_phase2 = np.hstack((A, np.eye(len(b))))  # Only original variables and slack variables
      c_phase2 = np.concatenate((original_c, np.zeros(len(b))))  # Extend original objective with zeros for slack variables

      # Recreate the tableau for Phase 2 without artificial variables
      Table_phase2 = np.zeros_like(Table_phase1)
      Table_phase2[0, :n_original + len(b)] = -c_phase2
      Table_phase2[1:, :n_original + len(b)] = Table_phase1[1:, :n_original + len(b)]
      Table_phase2[:, -1] = Table_phase1[:, -1]

      # Solve Phase 2
      num_iterations, solution, table = simplex(c_phase2, A_phase2, b)

      return num_iterations, solution, table

    if status == 1: ## ef Ax <= b
      return simplex(original_c, A, b)

c = np.array([10,-57,-9,-24])
A = np.array([[0.5,-5.5,-2.5,9],[0.5,-1.5,-0.5,1],[1,0,0,0]])
b = np.array([0,0,1])


#0 er  Ax >= b en 1 er Ax <= b
num_iterations, solution, table = two_phase_simplex(c, A, b, 1)
print("Fjöldi ítrana:", num_iterations)

print(table)

if solution is not None:
    print("Solution:", solution)

Fjöldi ítrana: 7
[[ 0. 30.  0. 42.  0. 18.  1.  1.]
 [ 0.  2.  0.  4.  1. -5.  2.  2.]
 [ 1.  0.  0.  0.  0.  0.  1.  1.]
 [ 0.  3.  1. -2.  0. -2.  1.  1.]]
Solution: [1. 0. 1. 0. 2. 0. 0.]


Nú er komið að því að endurbæta interior(c,A,b) fall sem á að virka fyrir prófin okkar.

Það fylgir Karmarkar aðferðinni sem lýsir sér svona:
https://en.wikipedia.org/wiki/Affine_scaling

Við fengum mjög frumstæðan kóða til að byrja með og það var nokkrir hlutir sem þurfti að laga.

Kóðinn sem við byrjuðum með ákvað engan sérstakan upphafspunkt sem hentaði verkefninu. Þess vegna bættum við við
setjum -Ix <= 0 og Ax <=b

In [21]:
import numpy as np
from scipy.optimize import linprog

# Set the dimensions as needed
m, n = 3, 2  # Adjust dimensions to match the problem

# Generate random A, x, b, and c
A = np.random.uniform(size=(m, n))
x = np.random.uniform(size=(n))
b = A @ x
c = np.random.uniform(size=(n))
x0 = x/2
##linprog


def interior(c, A, b, x):
    gamma = 0.7  # Control step size
    iter = 0
    while True:
        print("iter=",iter,x)
        v = b - A @ x
        if np.any(v <= 0) or np.any(x <= 0):
            print("Current point is not feasible.")
            return None
        D = np.diag(1.0 / v**2)
        try:
            hx = np.linalg.inv(A.T @ D @ A) @ c  # Direction improving c^T x
        except np.linalg.LinAlgError:
            print("Matrix inversion problem.")
            return None
        hv = -A @ hx
        if np.all(hv >= 0):
            print("Warning: Problem seems to be unbounded.")
            break
        ratio = -v[hv < 0] / hv[hv < 0]  # Compute ratios where hv is negative
        if ratio.size == 0:
            print("No positive step sizes; may be at or near optimum or problem infeasible.")
            break
        alpha = gamma * np.min(ratio[ratio > 0])
        x_new = x + alpha * hx
        if np.linalg.norm(x_new - x, np.inf) < 1e-6:
            break
        x = x_new
        iter = iter + 1
    return x

A_ = np.vstack((A,-np.eye(n)))
print(A_)
b_ = np.concatenate((b,np.zeros(n)))
result = interior(c, A_, b_, x0)
print("Feasible solution:", result)
res = linprog(c,A_eq = A,b_eq = b)
print("linprog=", res.x, A@res.x, b)

num_iterations, solution, table = two_phase_simplex(c, A, b, 1)

print("simplex=", solution)
print("simplex iterations =", num_iterations)


[[ 0.98545813  0.12190205]
 [ 0.99560654  0.93494084]
 [ 0.52139118  0.35377086]
 [-1.         -0.        ]
 [-0.         -1.        ]]
iter= 0 [0.1668566  0.39115594]
iter= 1 [0.20533194 0.74837162]
iter= 2 [0.20159352 0.87180889]
iter= 3 [0.219392  0.8886924]
iter= 4 [0.28215708 0.83260574]
iter= 5 [0.31837638 0.79634899]
iter= 6 [0.32764579 0.78808453]
iter= 7 [0.33190136 0.78397592]
iter= 8 [0.33302309 0.78296716]
iter= 9 [0.33350703 0.78250152]
iter= 10 [0.33363592 0.7823852 ]
iter= 11 [0.33369011 0.78233314]
iter= 12 [0.33370461 0.78232003]
iter= 13 [0.33371063 0.78231425]
iter= 14 [0.33371225 0.78231279]
Feasible solution: [0.33371225 0.78231279]
linprog= [0.3337132  0.78231188] [0.42422581 1.06366238 0.45075427] [0.42422581 1.06366238 0.45075427]
simplex= [3.33713200e-01 7.82311883e-01 0.00000000e+00 0.00000000e+00
 2.77555756e-17]
simplex iterations = 2
